In [1]:
import os
import pandas as pd
import nibabel as nib
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from skimage.measure import block_reduce
import numpy as np
from concurrent.futures import ProcessPoolExecutor
import time
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorly as tl

#Debugging import
import importlib
var = 'TensorDecisionTreeRegressorP' #the published version of code
package = importlib.import_module(var)
for name, value in package.__dict__.items():
    if not name.startswith("__"):
        globals()[name] = value

from TensorDecisionTreeRegressorP import *

import os
import nibabel as nib
import numpy as np
import matplotlib as plt
import pandas as pd
from sklearn.model_selection import train_test_split


# File path to the CSV file
csv_file = '/Users/zc56/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/ADNIData.csv'
df = pd.read_csv(csv_file)

# Remove rows where ADAS11_bl is missing (NaN)
df_cleaned = df.dropna(subset=['ADAS11_bl'])

# Extract the 'ADAS11_bl' column as the y variable
y_variable = df_cleaned['ADAS11_bl'].values

# Split the dataframe based on the DX_bl column values
cn_group = df_cleaned[df_cleaned['DX_bl'] == 'CN']
ad_group = df_cleaned[df_cleaned['DX_bl'] == 'AD']
lmci_group = df_cleaned[df_cleaned['DX_bl'] == 'LMCI']

# Display the counts for each group after removing NA
print(f"CN group size: {cn_group.shape[0]}")
print(f"AD group size: {ad_group.shape[0]}")
print(f"LMCI group size: {lmci_group.shape[0]}")

# Directory containing the 3D images
directory = '/Users/zc56/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/3D-Images/bl'

# Initialize dictionaries to hold the images and y values for each group
cn_images, ad_images, lmci_images = [], [], []
cn_y, ad_y, lmci_y = [], [], []

# Function to load the images based on PTID matching and append y values
def load_images_and_y(group, image_list, y_list):
    for _, row in group.iterrows():
        ptid = row['PTID']
        # Find the corresponding file based on PTID
        filename = f'{ptid}.nii.gz'
        file_path = os.path.join(directory, filename)
        
        if os.path.exists(file_path):
            # Load the NIfTI file
            img = nib.load(file_path)
            data = img.get_fdata()
            
            # Append the 3D image data and y value to the respective lists
            image_list.append(data)
            y_list.append(row['ADAS11_bl'])
        else:
            print(f"File {filename} not found.")

# Load images and y values for each group
load_images_and_y(cn_group, cn_images, cn_y)
load_images_and_y(ad_group, ad_images, ad_y)
load_images_and_y(lmci_group, lmci_images, lmci_y)

# Convert lists of 3D images and y values to NumPy arrays
if cn_images:
    cn_tensor = np.stack(cn_images, axis=0)
    cn_y = np.array(cn_y)
    print(f"CN 4D tensor shape: {cn_tensor.shape}")
    print(f"CN y shape: {cn_y.shape}")
else:
    print("No CN images loaded.")

if ad_images:
    ad_tensor = np.stack(ad_images, axis=0)
    ad_y = np.array(ad_y)
    print(f"AD 4D tensor shape: {ad_tensor.shape}")
    print(f"AD y shape: {ad_y.shape}")
else:
    print("No AD images loaded.")

if lmci_images:
    lmci_tensor = np.stack(lmci_images, axis=0)
    lmci_y = np.array(lmci_y)
    print(f"LMCI 4D tensor shape: {lmci_tensor.shape}")
    print(f"LMCI y shape: {lmci_y.shape}")
else:
    print("No LMCI images loaded.")


CN group size: 229
AD group size: 187
LMCI group size: 401
CN 4D tensor shape: (229, 48, 48, 48)
CN y shape: (229,)
AD 4D tensor shape: (187, 48, 48, 48)
AD y shape: (187,)
LMCI 4D tensor shape: (401, 48, 48, 48)
LMCI y shape: (401,)


In [2]:
X_train, X_test, y_train, y_test = train_test_split(lmci_tensor, lmci_y, test_size=0.2, random_state=42)


print(X_train.shape,y_train.shape)
model  =  TensorDecisionTreeRegressor(max_depth=6, min_samples_split=4,split_method='variance_LS', split_rank=4, CP_reg_rank=12, Tucker_reg_rank=12, n_mode=3)
model.use_mean_as_threshold  =  False
model.sample_rate  =  .5
X_coarsen_shape = (1,4,4,4)
X_coarsen_func = np.max
X_train_c = block_reduce(X_train,block_size=X_coarsen_shape, func=X_coarsen_func)
X_test_c = block_reduce(X_test,block_size=X_coarsen_shape, func=X_coarsen_func)
middle_z = X_train_c.shape[2] // 2
X_train_c = X_train_c[:,:,:,middle_z]
X_test_c = X_test_c[:,:,:,middle_z]
X_train_c = X_train_c+np.ones_like(X_train_c)*1e-3
print(X_train_c.shape,y_train.shape,X_test_c.shape)
model.fit(X_train_c,y_train)

predictions = model.predict(X_train_c,regression_method='mean')
print(f"mean train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='cp')
print(f"CP train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='tucker')
print(f"Tucker train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train)) 

predictions = model.predict(X_test_c,regression_method='mean')
print(f"mean test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='cp')
print(f"CP test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='tucker')
print(f"Tucker test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))

(320, 48, 48, 48) (320,)
(320, 12, 12) (320,) (81, 12, 12)
mean train RSE:  0.49534330634719587
CP train RSE:  0.0022288621342695914
Tucker train RSE:  0.0009497448616095284
mean test RSE:  1.565813031089088
CP test RSE:  3.5873792065584595
Tucker test RSE:  3.755752126718517


In [3]:
# Gradient Boosting Regressor Class
from GradientBoosting import *
# Initialize Gradient Boosting Regressor with a simple decision tree as weak learner
#weak_learner = TensorDecisionTreeRegressor(max_depth=3, min_samples_split=4, split_method='variance',split_rank=my_rank,n_mode=my_n_mode)
#weak_learner.use_mean_as_threshold = True
#weak_learner.sample_rate = 1.0
weak_learner = model
gradient_boosting_regressor = GradientBoostingRegressor(
    n_estimators=20,
    learning_rate=0.1,
    weak_learner=weak_learner
)
#gradient_boosting_regressor.pruning = True
# Fit a single tree model
weak_learner.fit(X_train_c, y_train)
weak_predictions = weak_learner.predict(X_test_c)


# Fit the Gradient Boosting model
gradient_boosting_regressor.fit(X_train_c, y_train, X_test_c, y_test)

fitting started
0 0 training RMSE === 7.737706822618623
0 0 testing RMSE === 12.996566526626966
0 1 training RMSE === 5.089903764622437
0 1 testing RMSE === 9.283254428248284
0 2 training RMSE === 3.209273023968686
0 2 testing RMSE === 6.682373109499321
0 3 training RMSE === 1.9061587152196653
0 3 testing RMSE === 4.778812791726211
0 4 training RMSE === 1.067546514427356
0 4 testing RMSE === 3.4896072321320477
0 5 training RMSE === 0.5513836012617743
0 5 testing RMSE === 2.6069779248447658
0 6 training RMSE === 0.3173098395858351
0 6 testing RMSE === 2.080529151064152
0 7 training RMSE === 0.2630594934991479
0 7 testing RMSE === 1.7933619842373365
0 8 training RMSE === 0.3561509254349389
0 8 testing RMSE === 1.7057151181285752
0 9 training RMSE === 0.5576546712729897
0 9 testing RMSE === 1.7542303101548022
0 10 training RMSE === 0.8282283569393364
0 10 testing RMSE === 1.8994492826923746
0 11 training RMSE === 1.1436765205613892
0 11 testing RMSE === 2.1419012879630075
0 12 training RM

In [2]:
from copy import deepcopy
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

# Gradient Boosting Regressor Class for LAD_TreeBoost
class LAD_TreeBoost:
    def __init__(self, n_estimators, learning_rate, weak_learner, n_iterations=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.weak_learner = weak_learner
        self.models = [deepcopy(self.weak_learner) for _ in range(n_estimators)]
        self.initial_model = None
        self.n_iterations = n_iterations
        self.pruning = False

    def fit(self, X, y, X_test=None, y_test=None):
        # Initialize the model with the median of y (LAD initialization)
        self.initial_model = self.initialize_model(y)
        current_pred = self.initial_model * np.ones(shape=y.shape)
        print("Fitting started")

        # Iteratively add weak learners
        for j in range(self.n_iterations):
            for i in range(self.n_estimators):
                # Calculate predictions of the current model
                current_pred = self.predict(X)

                # Calculate residual: pseudo-response for LAD is y - F_{m-1}(x)
                residual = y - current_pred
                print(f"Iteration {j}, Estimator {i}: Training LAD Loss = {np.mean(np.abs(residual))}")
                
                # Optional: Testing performance
                if X_test is not None and y_test is not None:
                    current_pred_test = self.predict(X_test)
                    residual_test = y_test - current_pred_test
                    print(f"Iteration {j}, Estimator {i}: Testing LAD Loss = {np.mean(np.abs(residual_test))}")

                # Fit weak learner to residual (pseudo-response)
                self.models[i].fit(X, residual)
                if self.pruning:
                    self.models[i].prune()

    def predict(self, X):
        # Start with initial model prediction (constant median value)
        y_pred = self.initial_model * np.ones(X.shape[0])

        # Add predictions from all weak learners
        for learner in self.models:
            learner_pred = learner.predict(X)
            if learner_pred is not None:
                # Update the prediction: Add gamma (median of predictions) for LAD
                y_pred += self.learning_rate * learner_pred
        
        return y_pred

    def initialize_model(self, y):
        # Initialize with the median of y for LAD
        return np.median(y)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(lmci_tensor, lmci_y, test_size=0.2, random_state=42)


print(X_train.shape,y_train.shape)
model  =  TensorDecisionTreeRegressor(max_depth=6, min_samples_split=4,split_method='variance_LS', split_rank=4, CP_reg_rank=12, Tucker_reg_rank=12, n_mode=3)
model.use_mean_as_threshold  =  False
model.sample_rate  =  .5
X_coarsen_shape = (1,4,4,4)
X_coarsen_func = np.max
X_train_c = block_reduce(X_train,block_size=X_coarsen_shape, func=X_coarsen_func)
X_test_c = block_reduce(X_test,block_size=X_coarsen_shape, func=X_coarsen_func)
middle_z = X_train_c.shape[2] // 2
X_train_c = X_train_c[:,:,:,middle_z]
X_test_c = X_test_c[:,:,:,middle_z]
print(X_train_c.shape,y_train.shape,X_test_c.shape)


noise_level = 1e-2
X_train_c = X_train_c+noise_level * np.random.randn(*X_train_c.shape)
X_test_c = X_test_c+noise_level * np.random.randn(*X_test_c.shape)
model.fit(X_train_c,y_train)

predictions = model.predict(X_train_c,regression_method='mean')
print(f"mean train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='cp')
print(f"CP train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='tucker')
print(f"Tucker train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train)) 

predictions = model.predict(X_test_c,regression_method='mean')
print(f"mean test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='cp')
print(f"CP test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='tucker')
print(f"Tucker test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))

(320, 48, 48, 48) (320,)
(320, 12, 12) (320,) (81, 12, 12)
mean train RSE:  0.36301627948486365
CP train RSE:  0.0013483275523180187
Tucker train RSE:  0.0007197489649219131
mean test RSE:  1.9043718496234767
CP test RSE:  2.7561558637589
Tucker test RSE:  2.977140839216029


In [12]:
weak_learner = model
gradient_boosting_regressor = LAD_TreeBoost(
    n_estimators=10,
    learning_rate=0.1,
    weak_learner=weak_learner
)
#gradient_boosting_regressor.pruning = True
# Fit a single tree model
weak_learner.fit(X_train_c, y_train)
weak_predictions = weak_learner.predict(X_test_c)

# Instantiate and train LAD_TreeBoost
lad_boost = LAD_TreeBoost(n_estimators=100, learning_rate=0.1, weak_learner=weak_learner)
lad_boost.fit(X_train, y_train, X_test, y_test)

# Make predictions on the test set
y_pred = lad_boost.predict(X_test)

# Calculate the Relative Absolute Error (RAE)
def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

rae_test = mean_absolute_error(y_test, y_pred) / np.mean(np.abs(y_test - np.median(y_test)))

print("Testing Relative Absolute Error (RAE):", rae_test)

Fitting started


IndexError: tuple index out of range